In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import poisson  # 泊松分布

In [2]:
# 英超使用(2018-2019)資料評估(2019-2020)賽季比分分布狀況
# TODO 資料整理
teams = pd.read_csv(os.getcwd() + '/data/82/teams.csv')
teams.set_index(["team_id"], inplace=True)

matches = pd.read_csv(os.getcwd() + '/data/82/matches.csv')
lastSeason = matches[matches['season_id'] == 8433]
curSeason = matches[matches['season_id'] == 9021]

# 主客隊最大進球數
homeMax = lastSeason['home_score'].max()
awayMax = lastSeason['home_score'].max()

# curSeason.set_index('match_id')
curSeason = curSeason.loc[curSeason.loc[:, "status"] == 8]
curSeason = curSeason.drop(columns=['home_half_score', 'home_red', 'home_yellow', 'home_corner','away_half_score', 'away_red', 'away_yellow', 'away_corner', 'season_id', 'season_name'])

In [3]:
# TODO 計算平均進球數
# 聯盟主隊平均主場進球數 = 主場賽季總進球數/比賽場數（賽季）
seasonHomeAvg = lastSeason['home_score'].sum() / lastSeason.shape[0]
# 聯盟主隊平均客場進球數 = 客場賽季總進球數/比賽場數（賽季）
seasonAwayAvg = lastSeason['away_score'].sum() / lastSeason.shape[0]

print('主場平均進球數: %s 球' % seasonHomeAvg)
print('客場平均進球數: %s 球' % seasonAwayAvg)

# TODO 統計資料
# 隊伍為主場平均進球
temp = lastSeason.groupby(['home_team'])['home_score', 'away_score'].sum()
temp.index.rename('team_id')
teams = teams.join(temp, on='team_id')
teams = teams.rename(columns={"home_score": "homeGetScore", "away_score": "homeLostScore"})

# 隊伍為主場總場次
temp = lastSeason.groupby(['home_team']).size()
temp.index.rename('team_id')
temp = temp.rename('homeMatchCount')
teams = teams.join(temp, on='team_id')

# 隊伍為客場平均進球
temp = lastSeason.groupby(['away_team'])['home_score', 'away_score'].sum()
temp.index.rename('team_id')
teams = teams.join(temp, on='team_id')
teams = teams.rename(columns={"home_score": "awayLostScore", "away_score": "awayGetScore"})

# 隊伍為客場總場次
temp = lastSeason.groupby(['away_team']).size()
temp.index.rename('team_id')
temp = temp.rename('awayMatchCount')
teams = teams.join(temp, on='team_id')

# 删除有NaN rows
teams = teams.dropna() 

主場平均進球數: 1.568421052631579 球
客場平均進球數: 1.2526315789473683 球


In [4]:
# TODO 計算進攻率(上賽季)
# 主隊主場平均進球數 = 主隊主場進球數 / 主場賽事場數。
# 主隊主場平均進球數 / 聯盟賽季主場平均進球數
teams['homeAttack'] =  teams['homeGetScore'] / teams['homeMatchCount'] / seasonHomeAvg
teams['awayAttack'] =  teams['awayGetScore'] / teams['awayMatchCount'] / seasonAwayAvg

# TODO 計算防守率(上賽季)
# 客隊客場平均失球數 = 客隊客場失球數 / 客場賽事場數。
# 客隊客場平均失球數 / 聯盟賽季客場平均失球數
teams['homeDefense'] =  teams['homeLostScore'] / teams['homeMatchCount'] / seasonAwayAvg
teams['awayDefense'] =  teams['awayLostScore'] / teams['awayMatchCount'] / seasonHomeAvg

In [5]:
teams

,Unnamed: 0,competition_id,short_name_zh,short_name_en,market_value,homeGetScore,homeLostScore,homeMatchCount,awayLostScore,awayGetScore,awayMatchCount,homeAttack,awayAttack,homeDefense,awayDefense
team_id,,,,,,,,,,,,,,,
10010,0,82,伯恩利,Burnley,198750000,24.0,32.0,19.0,36.0,21.0,19.0,0.805369,0.882353,1.344538,1.208054
10012,1,82,布莱顿,Brighton Hove Albion,219600000,19.0,28.0,19.0,32.0,16.0,19.0,0.637584,0.672269,1.176471,1.073826
10022,3,82,沃特福德,Watford,235150000,26.0,28.0,19.0,31.0,26.0,19.0,0.872483,1.092437,1.176471,1.040268
10115,4,82,伯恩茅斯,Bournemouth AFC,334250000,30.0,25.0,19.0,45.0,26.0,19.0,1.006711,1.092437,1.050420,1.510067
10135,5,82,曼联,Manchester United,753250000,33.0,25.0,19.0,29.0,32.0,19.0,1.107383,1.344538,1.050420,0.973154
10136,6,82,纽卡斯尔联,Newcastle United,273580000,24.0,25.0,19.0,23.0,18.0,19.0,0.805369,0.756303,1.050420,0.771812
10176,7,82,西汉姆联,West Ham United,348750000,32.0,27.0,19.0,28.0,20.0,19.0,1.073826,0.840336,1.134454,0.939597
10180,8,82,切尔西,Chelsea,807000000,39.0,12.0,19.0,27.0,24.0,19.0,1.308725,1.008403,0.504202,0.906040
10181,9,82,曼城,Manchester City,1280000000,57.0,12.0,19.0,11.0,38.0,19.0,1.912752,1.596639,0.504202,0.369128


In [6]:
# 整理當前賽季資料 準備進行預測
integration = pd.merge(curSeason, teams.loc[:,['homeAttack','homeDefense']], left_on='home_team', right_on='team_id')

integration = pd.merge(integration, teams.loc[:,['awayAttack','awayDefense']], left_on='away_team', right_on='team_id')

# # TODO 計算可能進球數
# 主隊可能進球數 = 主隊進攻率 * 客隊防守率 * 聯盟主隊平均主場進球數
integration['lambdaHome'] = integration['homeAttack'] * integration['awayDefense'] * seasonHomeAvg
# # 客隊可能進球數 = 客隊進攻率 * 主隊防守率 * 聯盟客隊平均客場進球數
integration['lambdaAway'] = integration['awayAttack'] * integration['homeDefense'] * seasonAwayAvg

teams.reset_index(inplace=True)

# # curSeason['awayPoisson'] = poisson.pmf(np.arange(awayMax + 1), lambdaAway)

# integration.loc[integration.loc[:, "match_id"] == 2004333]
integration

,Unnamed: 0,match_id,competition_id,status,open_date,home_team,away_team,home_rank,home_score,away_rank,away_score,homeAttack,homeDefense,awayAttack,awayDefense,lambdaHome,lambdaAway
0,6108,2679314,82,8,1566664200,10249,10215,1,3,2,1,1.845638,0.420168,1.302521,1.174497,3.399859,0.685537
1,6239,2679939,82,8,1575921600,10176,10215,16,1,11,3,1.073826,1.134454,1.302521,1.174497,1.978100,1.850951
2,6128,2679334,82,8,1568561400,10022,10215,20,2,7,2,0.872483,1.176471,1.302521,1.174497,1.607206,1.919505
3,6192,2679892,82,8,1573320600,10247,10215,3,2,5,0,0.805369,0.840336,1.302521,1.174497,1.483575,1.371075
4,6261,2679961,82,8,1577372400,10115,10215,14,1,11,1,1.006711,1.050420,1.302521,1.174497,1.854468,1.713843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,6264,2679964,82,8,1577372400,10250,10010,15,1,10,0,1.006711,0.882353,0.882353,1.208054,1.907453,0.975232
204,6101,2679307,82,8,1566747000,10226,10010,18,1,14,1,0.939597,0.882353,0.882353,1.208054,1.780290,0.975232
205,6292,2679992,82,8,1578754800,10180,10010,4,3,15,0,1.308725,0.504202,0.882353,1.208054,2.479689,0.557276
206,6338,2680038,82,8,1581769800,10182,10010,13,1,11,2,0.906040,1.260504,0.882353,1.208054,1.716708,1.393189


In [7]:
# TODO 使用Poisson進算比分分布
# result = {}

result = pd.DataFrame([], columns = ['matchId', 'openDate', 'home', 'away', 'score', 'mostLikely1', 'mostLikely2', 'mostLikely3', 'mostLikelyResult', 'LeastLikely1', 'LeastLikely2', 'LeastLikely3', 'LeastLikelyResult']) 

mlResultCount = {'True':0,'False':0}
llResultCount = {'True':0,'False':0}

for index, row in integration.iterrows():
    homePoisson = poisson.pmf(np.arange(homeMax + 1), row['lambdaHome'])
    awayPoisson = poisson.pmf(np.arange(awayMax + 1), row['lambdaAway'])

    scorePred =  {
        '主隊三球以上' : 0,
        '客隊三球以上' :0,
        '其他': 0
    }
    
    for i in np.arange(homeMax + 1):
        for j in np.arange(awayMax + 1):

            score = str(i) + ':' + str(j)
            
            if i > 3 and j > 3:
                score = '其他'
            elif i > 3:
                score = '主隊三球以上'
            elif j > 3:
                score = '客隊三球以上'

            prediction = homePoisson[i] * awayPoisson[j] * 100

            if not score == str(i) + ':' + str(j):
                prediction = scorePred[score] + prediction

            scorePred[score] = prediction
    
    score = str(row['home_score']) + ':' + str(row['away_score'])
    mostLikely = sorted(scorePred.items(), key=lambda obj: obj[1], reverse=True)
    LeastLikely = sorted(scorePred.items(), key=lambda obj: obj[1])
    mostLikelyList = [mostLikely[0][0], mostLikely[1][0], mostLikely[2][0]]
    LeastLikelyList = [LeastLikely[0][0], LeastLikely[1][0], LeastLikely[2][0]]

    mostLikelyResult = score in mostLikelyList
    LeastLikelyResult = not score in LeastLikelyList

    mlResultCount[str(mostLikelyResult)] += 1
    llResultCount[str(LeastLikelyResult)] += 1
    
    result = result.append(
        {
            'matchId': row['match_id'],
            'openDate': datetime.fromtimestamp(row['open_date']),
            'home': teams.loc[teams.loc[:, "team_id"] == row['home_team'], "short_name_zh"].values[0],
            'away': teams.loc[teams.loc[:, "team_id"] == row['away_team'], "short_name_zh"].values[0],
            'score': score,
            'mostLikely1': mostLikely[0][0],
            'mostLikely2': mostLikely[1][0],
            'mostLikely3': mostLikely[2][0],
            'mostLikelyResult': mostLikelyResult,
            'LeastLikely1': LeastLikely[0][0],
            'LeastLikely2': LeastLikely[1][0],
            'LeastLikely3': LeastLikely[2][0],
            'LeastLikelyResult': LeastLikelyResult
        }, 
        ignore_index=True)

In [8]:
# TODO 使用歷史資料進行驗證結果
result

,matchId,openDate,home,away,score,mostLikely1,mostLikely2,mostLikely3,mostLikelyResult,LeastLikely1,LeastLikely2,LeastLikely3,LeastLikelyResult
0,2679314,2019-08-25 00:30:00,利物浦,阿森纳,3:1,主隊三球以上,3:0,2:0,False,0:3,其他,客隊三球以上,True
1,2679939,2019-12-10 04:00:00,西汉姆联,阿森纳,1:3,主隊三球以上,客隊三球以上,1:1,False,其他,0:0,0:3,True
2,2679334,2019-09-15 23:30:00,沃特福德,阿森纳,2:2,客隊三球以上,1:1,1:2,False,其他,3:0,3:3,True
3,2679892,2019-11-10 01:30:00,莱斯特城,阿森纳,2:0,1:1,2:1,1:0,False,其他,3:3,0:3,True
4,2679961,2019-12-26 23:00:00,伯恩茅斯,阿森纳,1:1,主隊三球以上,1:1,2:1,True,其他,0:3,3:3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2679964,2019-12-26 23:00:00,埃弗顿,伯恩利,1:0,主隊三球以上,1:0,1:1,True,其他,0:3,3:3,True
204,2679307,2019-08-25 23:30:00,狼队,伯恩利,1:1,1:0,1:1,主隊三球以上,True,其他,3:3,0:3,True
205,2679992,2020-01-11 23:00:00,切尔西,伯恩利,3:0,主隊三球以上,2:0,3:0,True,其他,0:3,客隊三球以上,True
206,2680038,2020-02-15 20:30:00,南安普顿,伯恩利,1:2,1:1,2:1,主隊三球以上,False,其他,3:3,0:3,True


In [9]:
# TODO 最有可能比分成功預測結果
mlResultCount

{'True': 54, 'False': 154}

In [10]:
# TODO 最不可能比分成功預測結果
llResultCount

{'True': 199, 'False': 9}

In [11]:
# TODO 輸出結果
result.to_csv('result.csv', encoding='utf_8_sig')